# TensorFlow

**[MDC-01]** 必要なモジュールをインポートして、乱数のシードを設定します。

In [10]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data

np.random.seed(20160703)
tf.set_random_seed(20160703)

**[MDC-02]** MNISTのデータセットを用意します。

In [11]:
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


**[MDC-03]** フィルターに対応する Variable を用意して、入力データにフィルターとプーリング層を適用する計算式を定義します。

保存するときはモデルのパラメータを対応づけるため，変数(Variable)には名前をつけるのが良い．  
name=NoneのままだとしてもTFは自動的にuniquな名前をつけてくれるので，リストアするときは全く同じ式を立てれば問題ないが．．．

In [24]:
tf.get_default_graph()

In [25]:
graph = tf.Graph()

In [26]:
with graph.as_default():
    print(tf.get_default_graph())

In [27]:
tf.get_default_graph()

In [28]:
tf.get_default_graph()
num_filters = 16

x = tf.placeholder(tf.float32, [None, 784])
x_image = tf.reshape(x, [-1,28,28,1])  # N,H,W,C

# 変数に名前をつける
W_conv = tf.Variable(tf.truncated_normal([5,5,1,num_filters],  # FH,FW,C,FN    [filter_height, filter_width, in_channels, out_channels]
                                          stddev=0.1), name='W_conv')
"""
tf.get_variableは、既に存在すれば取得し(エラー出す?)、なければ変数を作成する関数です。
tf.Variableとは違い、変数値ではなく、第一引数に変数の名前を指定することが必須となっています。
このとき，initializerにはInitializer that generates~の関数しか使えない．
tf.truncated_normal は　initializer引数には使えないので注意
おそらく，関数名に ~_initializer()  が付いているのがinitializer引数に使える引数のよう
"""
#W_conv = tf.get_variable('W_conv', shape=[5,5,1,num_filters], 
#                           initializer=tf.truncated_normal_initializer(stddev=0.01))  
 
h_conv = tf.nn.conv2d(x_image, W_conv,
                      strides=[1,1,1,1], padding='SAME')
h_pool =tf.nn.max_pool(h_conv, ksize=[1,2,2,1],
                       strides=[1,2,2,1], padding='SAME')

**[MDC-04]** プーリング層からの出力を全結合層を経由してソフトマックス関数に入力する計算式を定義します。

In [4]:
h_pool_flat = tf.reshape(h_pool, [-1, 14*14*num_filters])

num_units1 = 14*14*num_filters
num_units2 = 1024

w2 = tf.Variable(tf.truncated_normal([num_units1, num_units2]), name='w2')
b2 = tf.Variable(tf.zeros([num_units2]), name='b2')
hidden2 = tf.nn.relu(tf.matmul(h_pool_flat, w2) + b2)

w0 = tf.Variable(tf.zeros([num_units2, 10]), name='w0')
b0 = tf.Variable(tf.zeros([10]), name='b0')
p = tf.nn.softmax(tf.matmul(hidden2, w0) + b0)

**[MDC-05]** 誤差関数 loss、トレーニングアルゴリズム train_step、正解率 accuracy を定義します。

In [5]:
t = tf.placeholder(tf.float32, [None, 10])
loss = -tf.reduce_sum(t * tf.log(p))
train_step = tf.train.AdamOptimizer(0.0005).minimize(loss)
correct_prediction = tf.equal(tf.argmax(p, 1), tf.argmax(t, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

**[MDC-06]** セッションを用意して、Variable を初期化します。

In [15]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())  # Variableを初期化する　これでVariableが値を持つことになる
saver = tf.train.Saver()  # セッション保存用  tf.global_variables_initializer()の実行後でなければならない

**[MDC-07]** パラメーターの最適化を4000回繰り返します。

最終的に、テストセットに対して約98%の正解率が得られます。

In [14]:
!rm -r  tensorflow

実行しているファイルが格納されているディレクトリのパスを取得したい。

https://qiita.com/neko_the_shadow/items/09ff3a423954a2adfe18

In [8]:
import os
cwd = os.getcwd()
cwd + '/hoge'

'/Users/yohei/Documents/Study/jupyter_tfbook/Chapter04/hoge'

In [9]:
__file__  # これはipnbだと使えないようだ?

NameError: name '__file__' is not defined

In [9]:
# pathlibを使って柔軟にファイル操作を行う
import pathlib

#  このスクリプトファイルがあるディレクトリの絶対パスを取得

root = pathlib.Path('.')  # 今のスクリプトファイルのPathオブジェクトを生成
abs_root = root.resolve()  # そのパスを絶対パスに変換
parent = abs_root.parent  # parent属性でこのスクリプトがあるディレクトリまでの絶対パスを作成
root, abs_root, parent

(PosixPath('.'),
 PosixPath('/Users/yohei/Documents/Study/jupyter_tfbook/Chapter04'),
 PosixPath('/Users/yohei/Documents/Study/jupyter_tfbook'))

In [11]:
str(abs_root / 'hoge')  # / でファイルをつなぐことができる

'/Users/yohei/Documents/Study/jupyter_tfbook/Chapter04/hoge'

In [12]:
cwd + '/hoge' == str(abs_root / 'hoge') 

True

In [16]:
i = 0
for _ in range(4000):
    i += 1
    batch_xs, batch_ts = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x: batch_xs, t: batch_ts})
    if i % 100 == 0:
        loss_val, acc_val = sess.run([loss, accuracy],
            feed_dict={x:mnist.test.images, t:mnist.test.labels})
        print ('Step: %d, Loss: %f, Accuracy: %f'
               % (i, loss_val, acc_val))
        # 現在のディレクトリにモデル用のディレクトリをつくらない場合は，今のスクリプトがあるところまでのpath(今だと pwd)を指定してやる必要がある. 
        # ディレクトリを作ってたる場合には directory/file だけでOK directory がない場合は作成してくれる
        saver.save(sess, 'tensorflow/mdc_session', global_step=i)  # ディレクトリを使う場合
        #　saver.save(sess, str(abs_root / 'mdc_session', global_step=i)  # 与えるファイルパスは './mdc_session'でもOK?

Step: 100, Loss: 2331.055176, Accuracy: 0.930300
Step: 200, Loss: 1905.531738, Accuracy: 0.941500
Step: 300, Loss: 1501.040894, Accuracy: 0.953800
Step: 400, Loss: 1300.982910, Accuracy: 0.960900
Step: 500, Loss: 1098.262695, Accuracy: 0.965800
Step: 600, Loss: 1033.074951, Accuracy: 0.969500
Step: 700, Loss: 1164.076050, Accuracy: 0.963200
Step: 800, Loss: 972.950317, Accuracy: 0.969500
Step: 900, Loss: 861.703735, Accuracy: 0.973500
Step: 1000, Loss: 941.364685, Accuracy: 0.970800
Step: 1100, Loss: 809.683899, Accuracy: 0.974800
Step: 1200, Loss: 713.957947, Accuracy: 0.977700
Step: 1300, Loss: 846.246338, Accuracy: 0.974200
Step: 1400, Loss: 721.964355, Accuracy: 0.978400
Step: 1500, Loss: 714.643005, Accuracy: 0.978300
Step: 1600, Loss: 779.952637, Accuracy: 0.976800
Step: 1700, Loss: 692.447815, Accuracy: 0.978300
Step: 1800, Loss: 801.884949, Accuracy: 0.974400
Step: 1900, Loss: 713.758789, Accuracy: 0.978900
Step: 2000, Loss: 701.541626, Accuracy: 0.980100
Step: 2100, Loss: 650.

**[MDC-08]** セッション情報を保存したファイルが生成されていることを確認します。

In [ ]:
!ls mdc_session*

In [ ]:
!rm mdc_session*.*

In [ ]:
cwd

In [ ]:
pwd+'/mdc_session'

# Keras

In [4]:
from keras.models import Sequential
from keras.layers.core import Activation, Lambda, Dense, Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.optimizers import Adam
from keras.models import Model
from keras import backend as K
from keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
from keras.initializers import TruncatedNormal, Zeros

In [5]:
np.random.seed(20160703)

In [6]:
model = Sequential()

In [7]:
num_filters, num_units =16, 1024
num_units1 = 14*14*num_filters
num_units2 = num_units

with K.name_scope('CNN'):
    model.add(Conv2D(filters=num_filters, kernel_size=5,
                     padding="same",
                     data_format="channels_last",
                     use_bias=False,
                     input_shape=(28, 28, 1),
                    kernel_initializer=TruncatedNormal(stddev=0.1),
                    name='convolution'
                    ))

    with K.name_scope('pooling'): # TensorFlowのname_scopeが使える． 色々とTensorFlowと変換性がある
        model.add(MaxPooling2D(pool_size=(2, 2),
                           padding="same", 
                           data_format="channels_last",
                            name='pool'))
        model.add(Flatten())  #  入力を平滑化する．バッチサイズに影響されない．

    model.add(Dense(units=num_units2, activation='relu', name='fully-connected'))
    model.add(Dense(units=10, activation='softmax', name='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
convolution (Conv2D)         (None, 28, 28, 16)        400       
_________________________________________________________________
pool (MaxPooling2D)          (None, 14, 14, 16)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 3136)              0         
_________________________________________________________________
fully-connected (Dense)      (None, 1024)              3212288   
_________________________________________________________________
softmax (Dense)              (None, 10)                10250     
Total params: 3,222,938
Trainable params: 3,222,938
Non-trainable params: 0
_________________________________________________________________


In [10]:
tensorboad = TensorBoard(log_dir='/tmp/mnist_df_logs', histogram_freq=1)  # histogram_freq=50で 50 epochごとに記録することになる
earlystopping = EarlyStopping(monitor='val_loss',  patience=5, verbose=1)

# ModelCheckpointでエポックごとにセッションを保存 : 詳しくは「詳解ディープラーニング  p300~」
checkpoint = ModelCheckpoint(
                            filepath=str(abs_root / 'keras_session/model_{epoch:02d}_vloss{val_loss:.3f}.hdf5'),
                            save_best_only=True)  #  save_best_only=True　だと そのとき最もよかった結果を上書きしないで済む?

In [11]:
model.compile(loss='categorical_crossentropy',
             optimizer=Adam(lr=0.0005),
             metrics=['accuracy'])

In [14]:
x_train, t_train = mnist.train.images,mnist.train.labels
x_train.shape, t_train.shape

((55000, 784), (55000, 10))

In [15]:
x_test, t_test = mnist.test.images,mnist.test.labels
x_test.shape, t_test.shape

((10000, 784), (10000, 10))

In [16]:
!mkdir keras_session  # Kerasの場合はディレクトリを作らないといけない

In [17]:
# 学習
x_train = x_train.reshape([-1,28,28,1])
x_test = x_test.reshape([-1,28,28,1])

hist = model.fit(x_train, t_train, batch_size=100,
                         epochs=7,
                         validation_data=(x_test, t_test),
                         callbacks=[checkpoint])

Train on 55000 samples, validate on 10000 samples
Epoch 1/7
55000/55000 [==============================] - 59s 1ms/step - loss: 0.2179 - acc: 0.9364 - val_loss: 0.0925 - val_acc: 0.9729
Epoch 2/7
55000/55000 [==============================] - 58s 1ms/step - loss: 0.0679 - acc: 0.9801 - val_loss: 0.0622 - val_acc: 0.9796
Epoch 3/7
55000/55000 [==============================] - 58s 1ms/step - loss: 0.0389 - acc: 0.9883 - val_loss: 0.0597 - val_acc: 0.9808
Epoch 4/7
55000/55000 [==============================] - 58s 1ms/step - loss: 0.0264 - acc: 0.9915 - val_loss: 0.0484 - val_acc: 0.9846
Epoch 5/7
55000/55000 [==============================] - 58s 1ms/step - loss: 0.0160 - acc: 0.9950 - val_loss: 0.0558 - val_acc: 0.9834
Epoch 6/7
55000/55000 [==============================] - 58s 1ms/step - loss: 0.0110 - acc: 0.9970 - val_loss: 0.0471 - val_acc: 0.9859
Epoch 7/7
55000/55000 [==============================] - 60s 1ms/step - loss: 0.0081 - acc: 0.9976 - val_loss: 0.0616 - val_acc: 0.981